In [23]:
import numpy as np
import scipy.io
from scipy import signal
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.metrics import (
    confusion_matrix,
    accuracy_score,
    recall_score,
    f1_score,
    ConfusionMatrixDisplay,
)
import matplotlib.pyplot as plt
import seaborn as sns
from braindecode.models import EEGNetv4
import copy
import random

seed = 102

torch.cuda.is_available()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.cuda.get_device_name(0)
torch.cuda.manual_seed(seed)
torch.manual_seed(seed)

In [24]:
def plot_learning_curves(train_losses, val_losses, train_accuracies, val_accuracies):
    epochs = range(1, len(train_losses) + 1)

    # Curva de Perda
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_losses, label="Perda Treinamento")
    plt.plot(epochs, val_losses, label="Perda Validação")
    plt.xlabel("Épocas")
    plt.ylabel("Loss")
    plt.legend()
    plt.title("Curva de Perda: Treino e validação")

    # Curva de Acurácia
    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_accuracies, label="Acurácia Treinamento")
    plt.plot(epochs, val_accuracies, label="Acurácia Validação")
    plt.xlabel("Épocas")
    plt.ylabel("Acurácia (%)")
    plt.legend()
    plt.title("Curva de Acurácia: Treino e validação")

    plt.tight_layout()
    plt.show()

In [25]:
def evaluate(model, test_loader, chanels):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.inference_mode():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Metrics
    accuracy = accuracy_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds, average="weighted")
    f1 = f1_score(all_labels, all_preds, average="weighted")

    cm = confusion_matrix(all_labels, all_preds)

    print(f"Test set Accuracy: {accuracy:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")

    classes = np.unique(np.concatenate((all_labels, all_preds)))

    # disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classes)
    # fig, ax = plt.subplots(figsize=(15, 15))  # aumenta a figura
    # disp.plot(ax=ax, cmap="Blues", xticks_rotation="vertical")  # rotaciona os rótulos para caber melhor
    # plt.show()

    print(all_labels)
    print(all_preds)

    return accuracy, recall, f1

In [ ]:
def bandpass_filter(
    dados, taxa_amostragem, freq_corte_low, freq_corte_high, ordem_filtro
):
    """





    Filtra dados EEG utilizando um filtro Butterworth passa-banda.






    Parâmetros:





    dados (ndarray): Dados do EEG com formato (número de eletrodos, número de amostras, número de frequências, número de trials).





    taxa_amostragem (int): Frequência de amostragem dos sinais EEG (Hz).





    freq_corte_low (float): Frequência de corte inferior do filtro passa-banda (Hz).





    freq_corte_high (float): Frequência de corte superior do filtro passa-banda (Hz).





    ordem_filtro (int): Ordem do filtro Butterworth.






    Retorna:





    ndarray: Dados EEG filtrados.





    """

    # **Construção do filtro passa-banda**

    # Cria o filtro passa-banda com os parâmetros especificados

    b, a = signal.butter(
        ordem_filtro,
        [freq_corte_low, freq_corte_high],
        btype="bandpass",
        analog=False,
        output="ba",
        fs=taxa_amostragem,
    )

    # **Filtragem dos dados**

    # Realiza o processo de filtragem para todas as frequências, trials e eletrodos

    num_eletrodos, num_amostras, num_freqs, num_trials = dados.shape

    filtered_data = np.zeros_like(dados)

    # Filtra os dados para cada frequência, trial e eletrodo

    for f in range(num_freqs):  # Para cada frequência de estimulação

        for trial in range(num_trials):  # Para cada trial

            for eletrodo in range(num_eletrodos):  # Para cada eletrodo

                # Filtra o sinal com o filtro de fase zero

                eletrodo_filtrado = signal.filtfilt(b, a, dados[eletrodo, :, f, trial])

                # Substitui o dado original pelo filtrado

                filtered_data[eletrodo, :, f, trial] = eletrodo_filtrado

    return filtered_data

In [27]:
def CCA_otimizacao(X, Y):

    # Calcula as linhas e colunas da matriz X
    linhas_X, colunas_X = X.shape
    # O número de amostras da matriz X é igual ao número de linhas de X
    num_amostras = linhas_X
    # Concatena X e Y
    V = np.concatenate((X, Y), axis=1)

    # Calcula a matriz S, matriz de covariância de X e Y.
    S = (1 / num_amostras) * (
        V.T @ V
        - (1 / num_amostras)
        * V.T
        @ np.ones((num_amostras, 1))
        @ np.ones((1, num_amostras))
        @ V
    )

    # Autocovariância de X
    Cxx = S[:colunas_X, :colunas_X]
    # Autocovariância de Y
    Cyy = S[colunas_X:, colunas_X:]
    # Covariância entre X e Y
    Cxy = S[:colunas_X, colunas_X:]

    # Calcula os autovalores e os autovetores de Cxx
    autovalores, autovetores = np.linalg.eig(Cxx)

    # Calcula a raiz quadrada dos autovalores
    raiz_autovalores = np.sqrt(autovalores)

    # Constrói a matriz diagonal dos autovalores
    raiz_lambda = np.diag(raiz_autovalores)

    # Calcula a inversa da matriz de autovetores
    inv_autovetores = np.linalg.inv(autovetores)

    # Calcula a raiz quadrada da matriz Cxx
    raiz_Cxx = np.dot(np.dot(autovetores, raiz_lambda), inv_autovetores)

    # Calcula a inversa da matriz raiz quadrada
    inv_raiz_Cxx = np.linalg.inv(raiz_Cxx)

    # Calcula os autovalores e os autovetores de Cyy
    autovalores, autovetores = np.linalg.eig(Cyy)

    # Calcula a raiz quadrada dos autovalores
    raiz_autovalores = np.sqrt(autovalores)

    # Constrói a matriz diagonal dos autovalores
    raiz_lambda = np.diag(raiz_autovalores)

    # Calcula a inversa da matriz de autovetores
    inv_autovetores = np.linalg.inv(autovetores)

    # Calcula a raiz quadrada da matriz Cyy
    raiz_Cyy = np.dot(np.dot(autovetores, raiz_lambda), inv_autovetores)

    # Calcula a inversa da matriz raiz quadrada
    inv_raiz_Cyy = np.linalg.inv(raiz_Cyy)

    # Calcula a matriz Kappa
    K = np.dot(inv_raiz_Cxx, np.dot(Cxy, inv_raiz_Cyy))

    # Decomposição da matriz Kappa, usando o método de decomposição em valores singulares
    Gamma, Lambda, Delta = np.linalg.svd(K)

    # Inversa da matriz Delta
    Delta = Delta.T

    # Calcula os combinadores lineares Wx e Wy.
    Wx = np.dot(inv_raiz_Cxx, Gamma[:, 0])
    Wy = np.dot(inv_raiz_Cyy, Delta[:, 0])

    correlation = Lambda[0]

    # Retorna os combinadores lineares.
    return Wx, Wy, correlation

In [ ]:
def matriz_referencia(
    numero_de_harmonicas, fase_inicial, sessoes, frequencia, fase, numero_de_amostras
):

    # Taxa de amostragem

    dt = 1 / 250

    # Número de amostras

    n = np.arange(numero_de_amostras)

    # Vetor de tempo

    t = dt * n

    y = []

    if fase_inicial == 0:

        theta = 0

    else:

        theta = fase

    # Gerando sinais senoidais e cossenoidais

    for k in range(1, numero_de_harmonicas + 1):

        y1 = np.sin(2 * np.pi * k * frequencia * t + theta)

        y2 = np.cos(2 * np.pi * k * frequencia * t + theta)

        y.append(y1)

        y.append(y2)

    # Transpõe o array Y
    y = np.array(y)
    y = np.transpose(y)

    # Repete o array para coincidir com o número de sessões

    Y = np.tile(y, (sessoes, 1))

    # Retorna a Matriz de sinais de referência

    return Y

In [7]:
def matriz_eeg(dados, sessoes_treino, eletrodos, indice_freq):
    # Inicializa a lista para armazenar os dados de treinamento
    X_treino = []

    # Itera sobre as sessões de treinamento
    for secao in sessoes_treino:
        # Adiciona os dados da sessão atual à lista
        X_treino.append(dados[eletrodos, :, indice_freq, secao])

    # Concatena os dados ao longo do eixo das colunas
    X_treino = np.concatenate(X_treino, axis=1)
    # Transpõe os dados para que cada linha represente uma amostra
    X_treino = np.transpose(X_treino)

    return X_treino

In [ ]:
def separar_em_janelas(matriz, tamanho_janela, incluir_ultimo=False):
    # Calcular o número total de linhas
    total_linhas = matriz.shape[0]

    # Calcular o número de grupos necessário
    num_janelas = total_linhas // tamanho_janela + (
        1 if total_linhas % tamanho_janela != 0 else 0
    )

    # Inicializar uma lista vazia para armazenar os grupos
    janelas = []

    # Iterar sobre a matriz, pegando as linhas de tamanho_grupo em tamanho_grupo e armazenando em grupos
    for i in range(0, total_linhas, tamanho_janela):
        janela = matriz[i : i + tamanho_janela]  # Pegar o grupo de tamanho_grupo linhas
        janelas.append(janela)  # Adicionar o grupo à lista

    # Se incluir_ultimo for False e houver linhas restantes, remover o último grupo
    if not incluir_ultimo and total_linhas % tamanho_janela != 0:
        janelas.pop()

    return janelas, num_janelas

In [ ]:
def train(
    model,
    train_loader,
    val_loader,
    criterion,
    optimizer,
    num_epochs=100,
    device=0,
    save_path="best_model_raw.pth",
):
    best_val_accuracy = 0.0
    model.to(device)
    train_losses, val_losses = [], []
    train_accuracies, val_accuracies = [], []
    for epoch in range(num_epochs):
        # Training Phase
        model.train()
        running_loss = 0.0
        train_correct = 0
        train_total = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

            # eval train
            _, preds = torch.max(outputs, 1)
            train_correct += (preds == labels).sum().item()
            train_total += labels.size(0)

        train_accuracy = train_correct / train_total
        avg_train_loss = running_loss / len(train_loader)

        # eval validation
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        with torch.inference_mode():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                # val accuracy
                _, preds = torch.max(outputs, 1)
                val_correct += (preds == labels).sum().item()
                val_total += labels.size(0)

        val_accuracy = val_correct / val_total
        avg_val_loss = val_loss / len(val_loader)

        train_losses.append(avg_train_loss)
        val_losses.append(avg_val_loss)
        train_accuracies.append(train_accuracy)
        val_accuracies.append(val_accuracy)

        # Save if best vall acc
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            best_model = copy.deepcopy(model.state_dict())
            torch.save(model.state_dict(), save_path)
            # print(f"Best model saved with accuracy: {best_val_accuracy:.4f}")

    #     print(
    #         f"Epoch {epoch + 1}/{num_epochs}: "
    #         f"Train Loss: {avg_train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, "
    #         f"Val Loss: {avg_val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}"
    #         )
    # plot_learning_curves(train_losses, val_losses, train_accuracies, val_accuracies)
    model.load_state_dict(best_model)
    return model

In [ ]:
frequencias_e_fases = scipy.io.loadmat(
    "C:/Users/machi/Documents/Mestrado/repos/data/benchmark/Freq_Phase.mat"
)


frequencias = frequencias_e_fases["freqs"]


frequencias = np.round(frequencias, 2).ravel()


fases = frequencias_e_fases["phases"]


filter_order = 10


freq_cut_high = 70


freq_cut_low = 6


sample_rate = 250


num_harmonica = 5


delay = 160  # 160 amostras, 0,5s (sem estimulação) + 0,14s (latencia para começo da evocação)

inform_fase = 0


tamanho_da_janela = 1


tamanho_da_janela = int(np.ceil(tamanho_da_janela * sample_rate))

In [14]:
frequencias[:8]

array([ 8.,  9., 10., 11., 12., 13., 14., 15.])

In [29]:
metricas_usuarios = []

for user in range(1, 2):
    file_path = f"C:/Users/machi/Documents/Mestrado/repos/data/benchmark/S{user}.mat"
    print(f"Processando Usuário {user}")

    # Carregar e preparar dados
    data = scipy.io.loadmat(file_path)["data"]
    data = bandpass_filter(data, sample_rate, freq_cut_low, freq_cut_high, filter_order)
    data = data[:, (delay) : (delay + 1250), :, :]

    occipital_electrodes = np.array([47, 53, 54, 55, 56, 57, 60, 61, 62])
    frequencias_desejadas = frequencias[:8]
    indices = [np.where(frequencias == freq)[0][0] for freq in frequencias_desejadas]

    num_canais, num_amostras, num_freqs, num_sections = data.shape

    metricas_crossval = []

    # Leave-one-session-out cross-validation
    for sessao_teste in range(6):  # 6 sessões no total
        sessoes_treino = [s for s in range(6) if s != sessao_teste]
        sessoes_teste = [sessao_teste]

        # Preparar os dados de treino
        Y_treino = np.zeros(
            (len(sessoes_treino) * num_amostras, num_harmonica * 2, len(indices))
        )
        for k in indices:
            y_treino = matriz_referencia(
                num_harmonica,
                inform_fase,
                len(sessoes_treino),
                frequencias[k],
                fases,
                num_amostras,
            )
            Y_treino[:, :, k] = y_treino

        X_treino = np.zeros(
            (
                len(sessoes_treino) * num_amostras,
                len(occipital_electrodes),
                len(indices),
            )
        )
        for k in range(len(indices)):
            x_treino = matriz_eeg(
                data, sessoes_treino, occipital_electrodes, indices[k]
            )
            X_treino[:, :, k] = x_treino

        # Preparar os dados de teste
        Y_teste = np.zeros(
            (len(sessoes_teste) * num_amostras, num_harmonica * 2, len(indices))
        )
        for k in indices:
            y_test = matriz_referencia(
                num_harmonica,
                inform_fase,
                len(sessoes_teste),
                frequencias[k],
                fases,
                num_amostras,
            )
            Y_teste[:, :, k] = y_test

        X_teste = np.zeros(
            (len(sessoes_teste) * num_amostras, len(occipital_electrodes), len(indices))
        )
        for k in range(len(indices)):
            x_test = matriz_eeg(data, sessoes_teste, occipital_electrodes, indices[k])
            X_teste[:, :, k] = x_test

        # Otimização CCA
        Combinadores_Y = []
        Combinadores_X = []
        correlacoes_max = []
        for k in range(len(indices)):
            Wx, Wy, corr = CCA_otimizacao(X_treino[:, :, k], Y_treino[:, :, k])
            Combinadores_Y.append(Wy)
            Combinadores_X.append(Wx)
            correlacoes_max.append(corr)

        Combinadores_X = np.column_stack(Combinadores_X)
        Combinadores_Y = np.column_stack(Combinadores_Y)

        # Separar em janelas
        X_teste_janelas = []
        X_treino_janelas = []
        Y_teste_janelas = []
        Y_treino_janelas = []

        for k in range(len(indices)):
            X_t, numero_janelas_teste = separar_em_janelas(
                X_teste[:, :, k], tamanho_da_janela, incluir_ultimo=False
            )
            Y_t, _ = separar_em_janelas(
                Y_teste[:, :, k], tamanho_da_janela, incluir_ultimo=False
            )

            X_v, numero_janelas_treino = separar_em_janelas(
                X_treino[:, :, k], tamanho_da_janela, incluir_ultimo=False
            )
            Y_v, _ = separar_em_janelas(
                Y_treino[:, :, k], tamanho_da_janela, incluir_ultimo=False
            )

            X_teste_janelas.append(X_t)
            Y_teste_janelas.append(Y_t)

            X_treino_janelas.append(X_v)
            Y_treino_janelas.append(Y_v)

        # Construir tensor de treinamento
        rotulos_treinamento = []
        tensor_treinamento = np.zeros(
            [5 * len(indices) * len(sessoes_treino), len(indices), tamanho_da_janela]
        )
        cont = 0

        for m in range(len(indices)):
            for j in range(numero_janelas_treino):
                janela_x = X_treino_janelas[m][j]
                rotulos_treinamento.append(frequencias[indices[m]])
                cont_1 = 0
                for w in range(len(indices)):
                    Wx = Combinadores_X[:, w]

                    projecao_x = np.dot(Wx, janela_x.T)
                    tensor_treinamento[cont, cont_1, :] = projecao_x

                    cont_1 += 1
                cont += 1

        rotulos_teste = []
        tensor_teste = np.zeros(
            [5 * len(indices) * len(sessoes_teste), len(indices), tamanho_da_janela]
        )
        cont = 0

        for m in range(len(indices)):

            for j in range(numero_janelas_teste):
                janela_x = X_teste_janelas[m][j]
                rotulos_teste.append(frequencias[indices[m]])
                cont_1 = 0

                for w in range(len(indices)):
                    Wx = Combinadores_X[:, w]

                    projecao_x = np.dot(Wx, janela_x.T)
                    tensor_teste[cont, cont_1, :] = projecao_x

                    cont_1 += 1
                cont += 1

        mapeamento = {
            rotulo: i for i, rotulo in enumerate(sorted(frequencias_desejadas))
        }

        rotulos_treinamento = torch.tensor(
            [mapeamento[rotulo.item()] for rotulo in rotulos_treinamento]
        )
        rotulos_teste = torch.tensor(
            [mapeamento[rotulo.item()] for rotulo in rotulos_teste]
        )

        X_treino = torch.tensor(tensor_treinamento, dtype=torch.float32).to(
            device
        )  # Converte para tensor float
        X_teste = torch.tensor(tensor_teste, dtype=torch.float32).to(device)
        Y_treino = torch.tensor(rotulos_treinamento, dtype=torch.long).to(
            device
        )  # Converte para tensor long (categorias)
        Y_teste = torch.tensor(rotulos_teste, dtype=torch.long).to(device)

        np.save(f"cca_subj_{user}_session_{sessao_teste}_train.npy", tensor_treinamento)
        np.save(
            f"cca_subj_{user}_session_{sessao_teste}_labels_train.npy", Y_treino.cpu()
        )
        np.save(f"cca_subj_{user}_session_{sessao_teste}_test.npy", tensor_teste)
        np.save(
            f"cca_subj_{user}_session_{sessao_teste}_labels_test.npy", Y_teste.cpu()
        )
        # Aqui você monta o modelo
        model = EEGNetv4(n_chans=40, n_outputs=40, n_times=250)
        model = model.to(device)

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=0.0001)

        dataset = TensorDataset(X_treino, Y_treino)
        train_size = int(0.85 * len(dataset))
        val_size = len(dataset) - train_size

        train_dataset, val_dataset = random_split(
            dataset,
            [train_size, val_size],
            generator=torch.Generator().manual_seed(seed),
        )

        train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)
        test_loader = DataLoader(
            TensorDataset(X_teste, Y_teste), batch_size=10, shuffle=False
        )

        # Treinar
        best_model = train(
            model, train_loader, val_loader, criterion, optimizer, num_epochs=1000
        )

        # Avaliar
        accuracy, recall, f1 = evaluate(
            best_model, test_loader, chanels=len(frequencias_desejadas)
        )

        metricas_crossval.append(
            {
                "usuario": user,
                "sessao_teste": sessao_teste,
                "acuracia": accuracy,
                "recall": recall,
                "f1-score": f1,
            }
        )
        print(
            f"Usuário {user} - Sessão {sessao_teste} Finalizada: Acurácia={accuracy:.4f}, Recall={recall:.4f}, F1={f1:.4f}"
        )

        # Salvar todas as sessões do usuário
    metricas_usuarios.extend(metricas_crossval)

    print("-" * 50)

# =====================================
# Mostrar resultados finais detalhados
# =====================================

print("\nResultados Finais Detalhados por Usuário e Sessão:")
print("=" * 50)
for resultado in metricas_usuarios:
    print(
        f"Usuário {resultado['usuario']} - Sessão {resultado['sessao_teste']}: "
        f"Acurácia = {resultado['acuracia']:.4f}, "
        f"Recall = {resultado['recall']:.4f}, "
        f"F1-Score = {resultado['f1-score']:.4f}"
    )

Processando Usuário 1


C:\Users\machi\AppData\Local\Temp\ipykernel_22612\2503681362.py:173: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Y_treino = torch.tensor(rotulos_treinamento, dtype=torch.long).to(
C:\Users\machi\AppData\Local\Temp\ipykernel_22612\2503681362.py:176: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Y_teste = torch.tensor(rotulos_teste, dtype=torch.long).to(device)


RuntimeError: Calculated padded input size per channel: (8 x 251). Kernel size: (40 x 1). Kernel size can't be greater than actual input size